# 1. Importar Librerias y llamar a la API

In [3]:
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
#esta libreria sirve para hacer el JSON mas bonito pero no funciona en notebook, mejor ir a labs
from IPython.display import JSON
import json
#esta libreria sirve para convertir la duracion de los videos a segundos
import isodate
import psycopg2

In [4]:
#guardar la llave en un documento .py llamado 'config' y llamar a la variable 'key'
from config import key

In [5]:
#La función se encuentra en https://github.com/googleapis/google-api-python-client/blob/main/docs/start.md
#escogí youtube pero hay mas opciones por revisar en supported APIs

from googleapiclient.discovery import build
youtubeanna = build('youtube', 'v3', developerKey = key)

In [6]:
#hacemos una variable que contenga el channel ID de las cuentas que queremos analizar
#se encuentran en click derecho view source Ctrl+f https://www.youtube.com/channel/ 

channel_ids = ['UCO8gWvBESNnIdZO6xyLLxTA', # autodinamico
               'UCUt99OukA2saWKgAG2Lb9OQ', # fergurquiza
               'UCSPwimlUyP62TPew_rG8vBg', # Jaime Gabaldoni
               'UCm7e2zwpwK5Sm7kzpGPWM6w', # manuelavasquez1
               'UCL5x3r5iEF7cnEgESDN0rsw', # gonzalo conducir
               'UC9EOEaPURNo0o1s0Inua3Yw', # Frankymostro
               'UCyYZafiawKepWzIMhCIRgNA', # Autocosmos México
               'UCgWyfYsBqo3qieiXs1ACbbw', # carwow América Latina
               'UC3FjkCViowUjmYldrssIMAQ' # chekomariscal
              ]

In [7]:
#creamos variables individuales en caso de necesitarlas mas adelante
autodinamico = 'UCO8gWvBESNnIdZO6xyLLxTA'
fer = 'UCUt99OukA2saWKgAG2Lb9OQ'
manuela = 'UCm7e2zwpwK5Sm7kzpGPWM6w'
cheko = 'UC3FjkCViowUjmYldrssIMAQ'
jaime ='UCSPwimlUyP62TPew_rG8vBg'
gonzalo = 'UCL5x3r5iEF7cnEgESDN0rsw'
franky ='UC9EOEaPURNo0o1s0Inua3Yw'
autocosmos = 'UCyYZafiawKepWzIMhCIRgNA'
carwow = 'UCgWyfYsBqo3qieiXs1ACbbw'

In [8]:
request = youtubeanna.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids))
    
response = request.execute()

In [9]:
response

{'kind': 'youtube#channelListResponse',
 'etag': 'k4GaK3NNFU8ocshDp29GiQX4AkY',
 'pageInfo': {'totalResults': 9, 'resultsPerPage': 5},
 'items': [{'kind': 'youtube#channel',
   'etag': 'eZjDx8KFjCB8y5nW9WPgGayXy-M',
   'id': 'UCm7e2zwpwK5Sm7kzpGPWM6w',
   'snippet': {'title': 'Manuela Vasquez',
    'description': 'Todos los Jueves y Domingos tenemos un nuevo video, donde reseñamos un auto, lo probamos, o tambien podemos conocerlo en su lanzamiento y ademas en ocaciones tenemos vlogs con autos o temas relacionados a ellos. \nEn fin, aquí hablamos de autos, y principalmente te ayudo a conocer el auto de el que quízas estas interesado a elegir.\n\n',
    'customUrl': '@manuelavasquez1',
    'publishedAt': '2007-11-07T22:58:31Z',
    'thumbnails': {'default': {'url': 'https://yt3.ggpht.com/ytc/AGIKgqOm7gO3xLdfE6qbouufFoPj9L_3Haprg1mH2q4Cfw=s88-c-k-c0x00ffffff-no-rj',
      'width': 88,
      'height': 88},
     'medium': {'url': 'https://yt3.ggpht.com/ytc/AGIKgqOm7gO3xLdfE6qbouufFoPj9L_3Ha

In [10]:
#creamos las funciones que nos van a ayudar a jalar los datos
def get_channel_stats(youtubeanna, channel_ids):
        
    all_data = []
    
    request = youtubeanna.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids))
    
    response = request.execute()

    for item in response['items']:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        
        all_data.append(data)
        
    return pd.DataFrame(all_data)

In [11]:
#creamos las funciones que nos van a ayudar a jalar los datos
def get_video_ids(youtubeanna, playlist_id):
    
    video_ids = []
    
    request = youtubeanna.playlistItems().list(
        part="snippet,contentDetails",
        playlistId=playlist_id,
        maxResults = 50
    )
    response = request.execute()
    
    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])
        
    next_page_token = response.get('nextPageToken')
    while next_page_token is not None:
        request = youtubeanna.playlistItems().list(
                    part='contentDetails',
                    playlistId = playlist_id,
                    maxResults = 50,
                    pageToken = next_page_token)
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')
        
    return video_ids
    

In [12]:
#creamos las funciones que nos van a ayudar a jalar los datos
def get_video_details(youtubeanna, video_ids):

    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtubeanna.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
    
    return pd.DataFrame(all_video_info)

In [13]:
#revisamos
channel_stats = get_channel_stats(youtubeanna, channel_ids)
channel_stats

,channelName,subscribers,views,totalVideos,playlistId
0,Frankymostro,341000,41086963,642,UU9EOEaPURNo0o1s0Inua3Yw
1,FERCHO URQUIZA,735000,57552328,256,UUUt99OukA2saWKgAG2Lb9OQ
2,carwow América Latina,1050000,215065002,761,UUgWyfYsBqo3qieiXs1ACbbw
3,AutoDinámico,1410000,442127611,1227,UUO8gWvBESNnIdZO6xyLLxTA
4,Manuela Vasquez,391000,84076280,623,UUm7e2zwpwK5Sm7kzpGPWM6w
5,Gonzalo. Mi Gusto es: ¡Conducir!,465000,110985445,1306,UUL5x3r5iEF7cnEgESDN0rsw
6,Chekomariscal,59800,1662318,214,UU3FjkCViowUjmYldrssIMAQ
7,Jaime Gabaldoni,275000,99568547,1207,UUSPwimlUyP62TPew_rG8vBg
8,Autocosmos México,542000,166328190,1150,UUyYZafiawKepWzIMhCIRgNA


In [14]:
playlist_autodinamico = channel_stats.loc[channel_stats['channelName']=='AutoDinámico', 'playlistId'].iloc[0]
playlist_autodinamico

'UUO8gWvBESNnIdZO6xyLLxTA'

In [15]:
# jalar las ids de todos los videos
video_ids = get_video_ids(youtubeanna, playlist_autodinamico)

In [16]:
request = youtubeanna.videos().list(
        part="snippet,contentDetails,statistics",
        id= video_ids[0:5])
    
response = request.execute()

In [17]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "etag": "AtvUbtj3UfUGJniZDcmBN62H9ms",
    "items": [
        {
            "contentDetails": {
                "caption": "false",
                "contentRating": {},
                "definition": "hd",
                "dimension": "2d",
                "duration": "PT7M6S",
                "licensedContent": true,
                "projection": "rectangular"
            },
            "etag": "ObgKchHANwJj0k1DDS-aSnR4bY4",
            "id": "PaqS3mRATdM",
            "kind": "youtube#video",
            "snippet": {
                "categoryId": "2",
                "channelId": "UCO8gWvBESNnIdZO6xyLLxTA",
                "channelTitle": "AutoDin\u00e1mico",
                "defaultAudioLanguage": "es",
                "description": "Gracias por ver nuestros videos :) Te invito a que me acompa\u00f1es a bordo de los autos que manejo en Instagram: @gabosalazar21\r\n\r\nTambi\u00e9n acomp\u00e1\u00f1anos alrededor del mundo para conocer m\u00e1s sobre la industria automotriz en 

In [74]:
# traemos los detalles de cada video
video_df = get_video_details(youtubeanna, video_ids)
video_df.head(3)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption
0,PaqS3mRATdM,AutoDinámico,La RAM más poderosa es ELÉCTRICA 😱 - RAM 1500 ...,Gracias por ver nuestros videos :) Te invito a...,"[gabo salazar, autodinamico, autos, coches, re...",2023-04-27T17:05:32Z,13708,965,None,28,PT7M6S,hd,false
1,yUy7KCvR0xQ,AutoDinámico,Kia EV9 - ¿Una Telluride eléctrica? | Primer V...,Gracias por ver nuestros videos :) Te invito a...,"[gabo salazar, autodinamico, autos, coches, re...",2023-04-26T23:57:44Z,22963,1140,None,26,PT6M50S,hd,false
2,tFrQC0icnEU,AutoDinámico,"Precios y versiones Honda Pilot 23' en MX 🚙🇯🇵,...",Gracias por ver nuestros videos :) Te invito a...,"[gabo salazar, autodinamico, autos, coches, re...",2023-04-24T23:05:44Z,25503,1161,None,31,PT5M42S,hd,false


# 2. Limpieza de datos

In [19]:
# checamos si hay nulos
video_df.isnull().any()

video_id          False
channelTitle      False
title             False
description       False
tags               True
publishedAt       False
viewCount         False
likeCount         False
favouriteCount     True
commentCount      False
duration          False
definition        False
caption           False
dtype: bool

In [20]:
# checamos los tipos de datos
video_df.dtypes

video_id          object
channelTitle      object
title             object
description       object
tags              object
publishedAt       object
viewCount         object
likeCount         object
favouriteCount    object
commentCount      object
duration          object
definition        object
caption           object
dtype: object

In [21]:
# convertimos las columnas a integers
numeric_cols = ['viewCount', 'likeCount', 'favouriteCount', 'commentCount']
video_df[numeric_cols] = video_df[numeric_cols].apply(pd.to_numeric, errors = 'coerce', axis = 1)

In [22]:
# convertimos las columnas a strings
string_cols = ['video_id', 'channelTitle', 'title', 'description', 'tags', 'definition', 'caption' ]
video_df[string_cols] = video_df[string_cols].astype(str)

In [23]:
# checamos si funciono
video_df.dtypes

video_id           object
channelTitle       object
title              object
description        object
tags               object
publishedAt        object
viewCount         float64
likeCount         float64
favouriteCount    float64
commentCount      float64
duration           object
definition         object
caption            object
dtype: object

In [25]:
# traemos el dia en que se publico el video
video_df['publishedAt'] = video_df['publishedAt'].apply(lambda x: parser.parse(x)) 
video_df['pushblishDayName'] = video_df['publishedAt'].apply(lambda x: x.strftime("%A")) 

In [26]:
# convertimos la duracion en segundos
#ES NECESARIO ----> import isodate 
video_df['durationSecs'] = video_df['duration'].apply(lambda x: isodate.parse_duration(x))
video_df['durationSecs'] = video_df['durationSecs'].astype('timedelta64[s]')

In [27]:
video_df[['durationSecs', 'duration']] 

,durationSecs,duration
0,426.0,PT7M6S
1,410.0,PT6M50S
2,342.0,PT5M42S
3,1223.0,PT20M23S
4,967.0,PT16M7S
...,...,...
1220,471.0,PT7M51S
1221,421.0,PT7M1S
1222,342.0,PT5M42S
1223,423.0,PT7M3S


In [28]:
# agregamos una columna que cuente los tags
video_df['tagCount'] = video_df['tags'].apply(lambda x: 0 if x is None else len(x))
video_df.head(3)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,pushblishDayName,durationSecs,tagCount
0,PaqS3mRATdM,AutoDinámico,La RAM más poderosa es ELÉCTRICA 😱 - RAM 1500 ...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-27 17:05:32+00:00,12857.0,935.0,NaN,28.0,PT7M6S,hd,false,Thursday,426.0,148
1,yUy7KCvR0xQ,AutoDinámico,Kia EV9 - ¿Una Telluride eléctrica? | Primer V...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-26 23:57:44+00:00,22804.0,1135.0,NaN,26.0,PT6M50S,hd,false,Wednesday,410.0,159
2,tFrQC0icnEU,AutoDinámico,"Precios y versiones Honda Pilot 23' en MX 🚙🇯🇵,...",Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-24 23:05:44+00:00,25484.0,1158.0,NaN,31.0,PT5M42S,hd,false,Monday,342.0,291


In [29]:
#agregamos una columna que extraiga el mes cuando se publico 
video_df['Month'] = pd.to_datetime(video_df['publishedAt']).dt.strftime('%b')
video_df.head(3)

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,pushblishDayName,durationSecs,tagCount,Month
0,PaqS3mRATdM,AutoDinámico,La RAM más poderosa es ELÉCTRICA 😱 - RAM 1500 ...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-27 17:05:32+00:00,12857.0,935.0,NaN,28.0,PT7M6S,hd,false,Thursday,426.0,148,Apr
1,yUy7KCvR0xQ,AutoDinámico,Kia EV9 - ¿Una Telluride eléctrica? | Primer V...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-26 23:57:44+00:00,22804.0,1135.0,NaN,26.0,PT6M50S,hd,false,Wednesday,410.0,159,Apr
2,tFrQC0icnEU,AutoDinámico,"Precios y versiones Honda Pilot 23' en MX 🚙🇯🇵,...",Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-24 23:05:44+00:00,25484.0,1158.0,NaN,31.0,PT5M42S,hd,false,Monday,342.0,291,Apr


In [30]:
#agregamos una columna que extraiga el año cuando se publico 
video_df['Year'] = pd.to_datetime(video_df['publishedAt']).dt.strftime('%Y')
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,pushblishDayName,durationSecs,tagCount,Month,Year
0,PaqS3mRATdM,AutoDinámico,La RAM más poderosa es ELÉCTRICA 😱 - RAM 1500 ...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-27 17:05:32+00:00,12857.0,935.0,NaN,28.0,PT7M6S,hd,false,Thursday,426.0,148,Apr,2023
1,yUy7KCvR0xQ,AutoDinámico,Kia EV9 - ¿Una Telluride eléctrica? | Primer V...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-26 23:57:44+00:00,22804.0,1135.0,NaN,26.0,PT6M50S,hd,false,Wednesday,410.0,159,Apr,2023
2,tFrQC0icnEU,AutoDinámico,"Precios y versiones Honda Pilot 23' en MX 🚙🇯🇵,...",Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-24 23:05:44+00:00,25484.0,1158.0,NaN,31.0,PT5M42S,hd,false,Monday,342.0,291,Apr,2023
3,u_Mor0gLUqk,AutoDinámico,Ford Mustang Mach E - Un excelente auto que se...,"Precios: $2,092,000 MXN\n\n¿A ti qué te pareci...","['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-21 15:07:16+00:00,129697.0,4368.0,NaN,347.0,PT20M23S,hd,false,Friday,1223.0,175,Apr,2023
4,Ikqq3eNHpW0,AutoDinámico,CR-V y Accord híbridos por fin llegan a México...,"Precios: \nCR-V Hybrid $899,900 MXN\nAccord Hy...","['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-19 23:44:25+00:00,96943.0,2990.0,NaN,371.0,PT16M7S,hd,false,Wednesday,967.0,204,Apr,2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,7ODJdEXqAQA,AutoDinámico,Audi A3 Sedán (2013 - 2021) - Lo mejor de Audi...,"(2013 -2021)\n\nRango de precios: $419,900 - $...","['autodinamico', 'opinión', 'review en español...",2014-12-30 01:11:29+00:00,871378.0,6461.0,NaN,383.0,PT7M51S,hd,false,Tuesday,471.0,295,Dec,2014
1221,QwU3LkTDrAQ,AutoDinámico,Volkswagen Passat (2011 - 2019) - Espacioso y ...,(2011 - 2019)\n\nMotor: 5 cilindros 2.5L \nPot...,"['autodinamico', 'opinión', 'review en español...",2014-12-18 16:44:08+00:00,208262.0,2096.0,NaN,158.0,PT7M1S,hd,false,Thursday,421.0,270,Dec,2014
1222,LsvCmFo1DRk,AutoDinámico,Nissan Sentra (2013-2019) - El familiar por ex...,Motor: 4 cilindros 1.8L 16V \nPotencia: 129 hp...,"['autodinamico', 'opinión', 'review en español...",2014-12-10 22:13:21+00:00,627355.0,4520.0,NaN,484.0,PT5M42S,hd,false,Wednesday,342.0,294,Dec,2014
1223,D7_7DcwkL-8,AutoDinámico,Seat Leon ⭐️ (2012-2020) - Un deportivo económico,Motor: 4 cilindros 1.4L TSI \nPotencia: 140 hp...,"['SEAT (Automobile Company)', 'SEAT León (Auto...",2014-12-10 17:30:40+00:00,391577.0,3590.0,NaN,525.0,PT7M3S,hd,false,Wednesday,423.0,405,Dec,2014


In [31]:
#agregamos que nos traiga los minutos 
video_df['Minutes'] = (video_df['durationSecs']/60).round()
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,pushblishDayName,durationSecs,tagCount,Month,Year,Minutes
0,PaqS3mRATdM,AutoDinámico,La RAM más poderosa es ELÉCTRICA 😱 - RAM 1500 ...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-27 17:05:32+00:00,12857.0,935.0,NaN,28.0,PT7M6S,hd,false,Thursday,426.0,148,Apr,2023,7.0
1,yUy7KCvR0xQ,AutoDinámico,Kia EV9 - ¿Una Telluride eléctrica? | Primer V...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-26 23:57:44+00:00,22804.0,1135.0,NaN,26.0,PT6M50S,hd,false,Wednesday,410.0,159,Apr,2023,7.0
2,tFrQC0icnEU,AutoDinámico,"Precios y versiones Honda Pilot 23' en MX 🚙🇯🇵,...",Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-24 23:05:44+00:00,25484.0,1158.0,NaN,31.0,PT5M42S,hd,false,Monday,342.0,291,Apr,2023,6.0
3,u_Mor0gLUqk,AutoDinámico,Ford Mustang Mach E - Un excelente auto que se...,"Precios: $2,092,000 MXN\n\n¿A ti qué te pareci...","['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-21 15:07:16+00:00,129697.0,4368.0,NaN,347.0,PT20M23S,hd,false,Friday,1223.0,175,Apr,2023,20.0
4,Ikqq3eNHpW0,AutoDinámico,CR-V y Accord híbridos por fin llegan a México...,"Precios: \nCR-V Hybrid $899,900 MXN\nAccord Hy...","['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-19 23:44:25+00:00,96943.0,2990.0,NaN,371.0,PT16M7S,hd,false,Wednesday,967.0,204,Apr,2023,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,7ODJdEXqAQA,AutoDinámico,Audi A3 Sedán (2013 - 2021) - Lo mejor de Audi...,"(2013 -2021)\n\nRango de precios: $419,900 - $...","['autodinamico', 'opinión', 'review en español...",2014-12-30 01:11:29+00:00,871378.0,6461.0,NaN,383.0,PT7M51S,hd,false,Tuesday,471.0,295,Dec,2014,8.0
1221,QwU3LkTDrAQ,AutoDinámico,Volkswagen Passat (2011 - 2019) - Espacioso y ...,(2011 - 2019)\n\nMotor: 5 cilindros 2.5L \nPot...,"['autodinamico', 'opinión', 'review en español...",2014-12-18 16:44:08+00:00,208262.0,2096.0,NaN,158.0,PT7M1S,hd,false,Thursday,421.0,270,Dec,2014,7.0
1222,LsvCmFo1DRk,AutoDinámico,Nissan Sentra (2013-2019) - El familiar por ex...,Motor: 4 cilindros 1.8L 16V \nPotencia: 129 hp...,"['autodinamico', 'opinión', 'review en español...",2014-12-10 22:13:21+00:00,627355.0,4520.0,NaN,484.0,PT5M42S,hd,false,Wednesday,342.0,294,Dec,2014,6.0
1223,D7_7DcwkL-8,AutoDinámico,Seat Leon ⭐️ (2012-2020) - Un deportivo económico,Motor: 4 cilindros 1.4L TSI \nPotencia: 140 hp...,"['SEAT (Automobile Company)', 'SEAT León (Auto...",2014-12-10 17:30:40+00:00,391577.0,3590.0,NaN,525.0,PT7M3S,hd,false,Wednesday,423.0,405,Dec,2014,7.0


In [32]:
#quitamos el .0 que tenian algunos datos
video_df['viewCount'] = video_df['viewCount'].apply(lambda x: int(x))
video_df['likeCount'] = video_df['likeCount'].apply(lambda x: int(x))
video_df['commentCount'] = video_df['commentCount'].apply(lambda x: int(x))
video_df['durationSecs'] = video_df['durationSecs'].apply(lambda x: int(x))
video_df

,video_id,channelTitle,title,description,tags,publishedAt,viewCount,likeCount,favouriteCount,commentCount,duration,definition,caption,pushblishDayName,durationSecs,tagCount,Month,Year,Minutes
0,PaqS3mRATdM,AutoDinámico,La RAM más poderosa es ELÉCTRICA 😱 - RAM 1500 ...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-27 17:05:32+00:00,12857,935,NaN,28,PT7M6S,hd,false,Thursday,426,148,Apr,2023,7.0
1,yUy7KCvR0xQ,AutoDinámico,Kia EV9 - ¿Una Telluride eléctrica? | Primer V...,Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-26 23:57:44+00:00,22804,1135,NaN,26,PT6M50S,hd,false,Wednesday,410,159,Apr,2023,7.0
2,tFrQC0icnEU,AutoDinámico,"Precios y versiones Honda Pilot 23' en MX 🚙🇯🇵,...",Gracias por ver nuestros videos :) Te invito a...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-24 23:05:44+00:00,25484,1158,NaN,31,PT5M42S,hd,false,Monday,342,291,Apr,2023,6.0
3,u_Mor0gLUqk,AutoDinámico,Ford Mustang Mach E - Un excelente auto que se...,"Precios: $2,092,000 MXN\n\n¿A ti qué te pareci...","['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-21 15:07:16+00:00,129697,4368,NaN,347,PT20M23S,hd,false,Friday,1223,175,Apr,2023,20.0
4,Ikqq3eNHpW0,AutoDinámico,CR-V y Accord híbridos por fin llegan a México...,"Precios: \nCR-V Hybrid $899,900 MXN\nAccord Hy...","['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-19 23:44:25+00:00,96943,2990,NaN,371,PT16M7S,hd,false,Wednesday,967,204,Apr,2023,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,7ODJdEXqAQA,AutoDinámico,Audi A3 Sedán (2013 - 2021) - Lo mejor de Audi...,"(2013 -2021)\n\nRango de precios: $419,900 - $...","['autodinamico', 'opinión', 'review en español...",2014-12-30 01:11:29+00:00,871378,6461,NaN,383,PT7M51S,hd,false,Tuesday,471,295,Dec,2014,8.0
1221,QwU3LkTDrAQ,AutoDinámico,Volkswagen Passat (2011 - 2019) - Espacioso y ...,(2011 - 2019)\n\nMotor: 5 cilindros 2.5L \nPot...,"['autodinamico', 'opinión', 'review en español...",2014-12-18 16:44:08+00:00,208262,2096,NaN,158,PT7M1S,hd,false,Thursday,421,270,Dec,2014,7.0
1222,LsvCmFo1DRk,AutoDinámico,Nissan Sentra (2013-2019) - El familiar por ex...,Motor: 4 cilindros 1.8L 16V \nPotencia: 129 hp...,"['autodinamico', 'opinión', 'review en español...",2014-12-10 22:13:21+00:00,627355,4520,NaN,484,PT5M42S,hd,false,Wednesday,342,294,Dec,2014,6.0
1223,D7_7DcwkL-8,AutoDinámico,Seat Leon ⭐️ (2012-2020) - Un deportivo económico,Motor: 4 cilindros 1.4L TSI \nPotencia: 140 hp...,"['SEAT (Automobile Company)', 'SEAT León (Auto...",2014-12-10 17:30:40+00:00,391577,3590,NaN,525,PT7M3S,hd,false,Wednesday,423,405,Dec,2014,7.0


In [75]:
#cambiamos el titulo de algunas columnas
video_df.rename(columns = {'channelTitle':'Channel', 'publishedAt':'Published','viewCount':'Views','likeCount':'Likes','commentCount':'Comments','pushblishDayName':'Day_Pushblished','durationSecs':'Duration_Secs'}, inplace = True)
video_df.rename(columns = {'video_id':'Video_Id', 'title':'Title','description':'Description','tags':'Tags','definition':'Definition','caption':'Caption','tagCount':'Tag_Count', 'title_no_stopwords':'Title_No_Stopwords','favouriteCount':'Favourites','duration':'Duration' }, inplace = True)
video_df.head(3)

,Video_Id,Channel,Title,Description,Tags,Published,Views,Likes,Favourites,Comments,Duration,Definition,Caption
0,PaqS3mRATdM,AutoDinámico,La RAM más poderosa es ELÉCTRICA 😱 - RAM 1500 ...,Gracias por ver nuestros videos :) Te invito a...,"[gabo salazar, autodinamico, autos, coches, re...",2023-04-27T17:05:32Z,13708,965,None,28,PT7M6S,hd,false
1,yUy7KCvR0xQ,AutoDinámico,Kia EV9 - ¿Una Telluride eléctrica? | Primer V...,Gracias por ver nuestros videos :) Te invito a...,"[gabo salazar, autodinamico, autos, coches, re...",2023-04-26T23:57:44Z,22963,1140,None,26,PT6M50S,hd,false
2,tFrQC0icnEU,AutoDinámico,"Precios y versiones Honda Pilot 23' en MX 🚙🇯🇵,...",Gracias por ver nuestros videos :) Te invito a...,"[gabo salazar, autodinamico, autos, coches, re...",2023-04-24T23:05:44Z,25503,1161,None,31,PT5M42S,hd,false


In [62]:
#eliminamos columnas que no sean necesarias
video_df_clean = video_df.drop(['Favourites','Caption','Description'], axis=1)
video_df_clean

,Video_Id,Channel,Title,Tags,Published,Views,Likes,Comments,Duration,Definition,Day_Pushblished,Duration_Secs,Tag_Count,Month,Year,Minutes
0,PaqS3mRATdM,AutoDinámico,La RAM más poderosa es ELÉCTRICA 😱 - RAM 1500 ...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-27 17:05:32+00:00,12857,935,28,PT7M6S,hd,Thursday,426,148,Apr,2023,7.0
1,yUy7KCvR0xQ,AutoDinámico,Kia EV9 - ¿Una Telluride eléctrica? | Primer V...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-26 23:57:44+00:00,22804,1135,26,PT6M50S,hd,Wednesday,410,159,Apr,2023,7.0
2,tFrQC0icnEU,AutoDinámico,"Precios y versiones Honda Pilot 23' en MX 🚙🇯🇵,...","['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-24 23:05:44+00:00,25484,1158,31,PT5M42S,hd,Monday,342,291,Apr,2023,6.0
3,u_Mor0gLUqk,AutoDinámico,Ford Mustang Mach E - Un excelente auto que se...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-21 15:07:16+00:00,129697,4368,347,PT20M23S,hd,Friday,1223,175,Apr,2023,20.0
4,Ikqq3eNHpW0,AutoDinámico,CR-V y Accord híbridos por fin llegan a México...,"['gabo salazar', 'autodinamico', 'autos', 'coc...",2023-04-19 23:44:25+00:00,96943,2990,371,PT16M7S,hd,Wednesday,967,204,Apr,2023,16.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1220,7ODJdEXqAQA,AutoDinámico,Audi A3 Sedán (2013 - 2021) - Lo mejor de Audi...,"['autodinamico', 'opinión', 'review en español...",2014-12-30 01:11:29+00:00,871378,6461,383,PT7M51S,hd,Tuesday,471,295,Dec,2014,8.0
1221,QwU3LkTDrAQ,AutoDinámico,Volkswagen Passat (2011 - 2019) - Espacioso y ...,"['autodinamico', 'opinión', 'review en español...",2014-12-18 16:44:08+00:00,208262,2096,158,PT7M1S,hd,Thursday,421,270,Dec,2014,7.0
1222,LsvCmFo1DRk,AutoDinámico,Nissan Sentra (2013-2019) - El familiar por ex...,"['autodinamico', 'opinión', 'review en español...",2014-12-10 22:13:21+00:00,627355,4520,484,PT5M42S,hd,Wednesday,342,294,Dec,2014,6.0
1223,D7_7DcwkL-8,AutoDinámico,Seat Leon ⭐️ (2012-2020) - Un deportivo económico,"['SEAT (Automobile Company)', 'SEAT León (Auto...",2014-12-10 17:30:40+00:00,391577,3590,525,PT7M3S,hd,Wednesday,423,405,Dec,2014,7.0


In [63]:
#revisamos los tipos de datos para asi armar la tabla en SQL
video_df_clean.dtypes

Video_Id                            object
Channel                             object
Title                               object
Tags                                object
Published          datetime64[ns, tzutc()]
Views                                int64
Likes                                int64
Comments                             int64
Duration                            object
Definition                          object
Day_Pushblished                     object
Duration_Secs                        int64
Tag_Count                            int64
Month                               object
Year                                object
Minutes                            float64
dtype: object

# 3. Paso 3: Crear la tabla de destino en redshift

In [36]:
#entrar a DBeaver y crear New Database connection
#ingresar Usuario, Contraseña, Host, port y Database
#ir a tu usuario y crear un nuevo script SQL

CREATE TABLE autodinamico (
Video_Id VARCHAR NOT NULL,  	
Channel	VARCHAR NOT NULL,  	
Title VARCHAR(700) NOT NULL,	
Tags VARCHAR(700) NOT NULL,	
Published DATE NOT NULL,  	
Views INT not null,
Likes INT not null,	
Comments INT not null,	
Duration VARCHAR NOT NULL, 	
Definition VARCHAR NOT NULL, 
Day_Pushblished	VARCHAR NOT NULL, 
Duration_Secs INT not null,
Tag_Count INT not null,
Month VARCHAR NOT NULL, 	
Year VARCHAR NOT NULL, 	
Minutes FLOAT not null,
PRIMARY KEY (Video_Id),
UNIQUE (Video_Id)
);

# Paso 4: establecer conexion a Redshift

In [37]:
from config import redshift_pwd
from config import user
from config import host
from config import data_base

In [69]:
try:
    conn = psycopg2.connect(
        host= host,
        dbname=data_base,
        user=user,
        password=redshift_pwd,
        port='5439'
    )
    print("Connected to Redshift successfully!")
    
except Exception as e:
    print("Unable to connect to Redshift.")
    print(e)

Connected to Redshift successfully!


# Paso 5: Verificar que la tabla exista

In [70]:
cur = conn.cursor()
#ejecutar una query para verificar si existe la tabla que creamos
cur.execute("SELECT * FROM autodinamico")
# traer los resultados
results = cur.fetchall()
results

[]

# Paso 6: Enviar resultados de tabla a redshift

In [71]:
#llamamos a la funcion dentro de extras que trae valores, tambn lo hace sqlalquemy pero puede haber errores
#nos permite hacer un INSERT multiple con respecto a las columnas del Dataframe
from psycopg2.extras import execute_values

#establecer el cursor hacia la base de datos
cur = conn.cursor()

#ponemos el nombre de la tabla a la cual queremos insertar los datos
table_name = 'autodinamico'

#ponemos las columnas 
columns = ['Video_Id', 'Channel', 'Title','Tags', 'Published', 'Views',
           'Likes', 'Comments', 'Duration', 'Definition', 'Day_Pushblished', 'Duration_Secs',
           'Tag_Count', 'Month', 'Year', 'Minutes']

#sacamos los resultados que entran a una matriz en numpy y hace un ciclo para insertar los datos
#es una lista de tuplas
#cada tupla es una lista del data frame
values = [tuple(x) for x in video_df_clean.to_numpy()]

#aqui inserta los datos separados por comas y le estamos diciendo que es un string
insert_sql = f"INSERT INTO {table_name} ({', '.join(columns)}) VALUES %s"

#comenzar la transaccion que necesita un inicio y una autorizacion 
#te pide el cursor, la query y los valores
cur.execute("BEGIN")
execute_values(cur, insert_sql, values)
cur.execute("COMMIT")


In [72]:
#nos vamos a DBeaver y checamos con un SELECT * para ver si funciono

In [73]:
# SIEMPRE HAY QUE CERRAR EL CURSOR Y LA CONEXION!!!!
cur.close()
conn.close()